In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"
import plotly.express as px
from datetime import datetime

In [21]:
df = pd.read_csv('rfm_data.csv')
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [22]:
missing = df.isnull().sum()
print(missing)

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64


In [23]:
df.dtypes

CustomerID              int64
PurchaseDate           object
TransactionAmount     float64
ProductInformation     object
OrderID                 int64
Location               object
dtype: object

In [24]:
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])

In [25]:
df.dtypes

CustomerID                     int64
PurchaseDate          datetime64[ns]
TransactionAmount            float64
ProductInformation            object
OrderID                        int64
Location                      object
dtype: object

In [26]:
#calculate Recency
df['Recency'] = (datetime.now().date() - df['PurchaseDate'].dt.date).dt.days

#calculate Frequency
frequency_data = df.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns = {'OrderID' : 'Frequency'}, inplace = True)
df = df.merge(frequency_data, on='CustomerID', how='left')

#calculate monetary
monetary_data = df.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns = {'TransactionAmount' : 'MonetaryValue'}, inplace = True)
df = df.merge(monetary_data, on='CustomerID', how='left')

print(df.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  Recency  Frequency  MonetaryValue  
0     Tokyo      259          1         943.31  
1    London      259          1         463.70  
2  New York      259          1          80.28  
3    London      259          1         221.29  
4     Paris      259          1         739.56  


In [28]:
recency_score = [5,4,3,2,1]
frequency_score = [1,2,3,4,5]
monetary_score = [1,2,3,4,5]

df['Recency_Score'] = pd.cut(df['Recency'], bins=5, labels = recency_score)
df['Frequency_Score'] = pd.cut(df['Frequency'], bins=5, labels = frequency_score)
df['Monetary_Score'] = pd.cut(df['MonetaryValue'], bins=5, labels = monetary_score)




0    1
1    1
2    1
3    1
4    1
Name: Recency_Score, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]


In [29]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,Recency_Score,Frequency_Score,Monetary_Score
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,259,1,943.31,1,1,2
1,2188,2023-04-11,463.70,Product A,176819,London,259,1,463.70,1,1,1
2,4608,2023-04-11,80.28,Product A,340062,New York,259,1,80.28,1,1,1
3,2559,2023-04-11,221.29,Product A,239145,London,259,1,221.29,1,1,1
4,9482,2023-04-11,739.56,Product A,194545,Paris,259,1,739.56,1,1,2


In [30]:
df['Recency_Score'] = df['Recency_Score'].astype(int)
df['Frequency_Score'] = df['Frequency_Score'].astype(int)
df['Monetary_Score'] = df['Monetary_Score'].astype(int)


In [31]:
df['RFM_Score'] = df['Recency_Score'] + df['Frequency_Score'] + df['Monetary_Score']

segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
df['Value_Segment'] = pd.qcut(df['RFM_Score'], q=3, labels=segment_labels)

In [32]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,Recency_Score,Frequency_Score,Monetary_Score,RFM_Score,Value_Segment
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,259,1,943.31,1,1,2,4,Low-Value
1,2188,2023-04-11,463.70,Product A,176819,London,259,1,463.70,1,1,1,3,Low-Value
2,4608,2023-04-11,80.28,Product A,340062,New York,259,1,80.28,1,1,1,3,Low-Value
3,2559,2023-04-11,221.29,Product A,239145,London,259,1,221.29,1,1,1,3,Low-Value
4,9482,2023-04-11,739.56,Product A,194545,Paris,259,1,739.56,1,1,2,4,Low-Value


In [38]:
segment_counts = df['Value_Segment'].value_counts().reset_index()
segment_counts.columns = ['Value_Segment', 'Count']

pastel_colors = px.colors.qualitative.Pastel

fig_segment_dist = px.bar(segment_counts, x='Value_Segment', y='Count', 
                          color='Value_Segment', color_discrete_sequence = pastel_colors,
                          title='RFM Value Segment Distribution')

fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Count',
                              showlegend=False)

fig_segment_dist.show()

In [42]:
df['RFM Customer Segments'] = ''

df.loc[df['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
df.loc[(df['RFM_Score'] >= 6) & (df['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
df.loc[(df['RFM_Score'] >= 5) & (df['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
df.loc[(df['RFM_Score'] >= 4) & (df['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
df.loc[(df['RFM_Score'] >= 3) & (df['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

print(df[['CustomerID', 'RFM Customer Segments']])

     CustomerID RFM Customer Segments
0          8814            Can't Lose
1          2188                  Lost
2          4608                  Lost
3          2559                  Lost
4          9482            Can't Lose
..          ...                   ...
995        2970   Potential Loyalists
996        6669   Potential Loyalists
997        8836   Potential Loyalists
998        1440   Potential Loyalists
999        4759   Potential Loyalists

[1000 rows x 2 columns]


In [48]:
segment_product_counts = df.groupby(['Value_Segment', 'RFM Customer Segments']).size().reset_index(name='Count')

#print(segment_product_counts)
segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

fig_treemap_segment_product = px.treemap(segment_product_counts, 
                                         path=['Value_Segment', 'RFM Customer Segments'], 
                                         values='Count',
                                         color='Value_Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

In [50]:
champions_segment = df[df['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['Recency_Score'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['Frequency_Score'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['Monetary_Score'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

fig.show()

In [51]:
correlation_matrix = champions_segment[['Recency_Score', 'Frequency_Score', 'Monetary_Score']].corr()

# Visualize the correlation matrix using a heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()

In [54]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = df['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
champions_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

In [57]:
segment_scores = df.groupby('RFM Customer Segments')['Recency_Score', 'Frequency_Score', 'Monetary_Score'].mean().reset_index()

# Create a grouped bar chart to compare segment scores
fig = go.Figure()

# Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['Recency_Score'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

# Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['Frequency_Score'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

# Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['Monetary_Score'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

# Update the layout
fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()

/var/folders/kk/l7qpv_zs0q1b1zqj_4frh_dh0000gn/T/ipykernel_78312/4218318092.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

